In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import requests as re


In [2]:
listado_urls = pd.read_csv("../FASES/Files/listado_urls.csv")
listado_urls

,URL
0,https://www.vinissimus.com/es/vino/ultreia-sai...
1,https://www.vinissimus.com/es/vino/jose-parien...
2,https://www.vinissimus.com/es/vino/tinto-pesqu...
3,https://www.vinissimus.com/es/vino/la-montesa/
4,https://www.vinissimus.com/es/vino/terras-gauda/
...,...
819,https://www.vinissimus.com/es/vino/capricho-va...
820,https://www.vinissimus.com/es/vino/campo-viejo...
821,https://www.vinissimus.com/es/vino/poesia-garn...
822,https://www.vinissimus.com/es/vino/monasterio-...


In [9]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Suponiendo que ya tienes un DataFrame llamado listado_urls con una columna 'URLs'

# Obtener la lista de URLs del DataFrame
urls_list = listado_urls['URL'].tolist()

# urls_list = [
#             'https://www.vinissimus.com/es/vino/dominio-de-tares-cepas-viejas/',
#             'https://www.vinissimus.com/es/vino/beronia-reserva/',
#             'https://www.vinissimus.com/es/vino/torre-la-moreira/',
#             'https://www.vinissimus.com/es/vino/albarino-de-fefinanes/', 
#             'https://www.vinissimus.com/es/vino/vega-sicilia-unico/', 
#             'https://www.vinissimus.com/es/vino/lanzaga/', 
#             'https://www.vinissimus.com/es/vino/numanthia/', 
#             'https://www.vinissimus.com/es/vino/vina-bosconia-reserva/', 
#             'https://www.vinissimus.com/es/vino/do-ferreiro/'
# ]

# Bucle a través de cada URL en la lista

dict_results = {
    "name_w": [], "year_w": [], "price": [], "user_score": [], "type_w": [], "region": [], 
     "alcohol_content": [], "wine_grape": [], "pk_score": [], "pn_score": [],
    "sk_score":[]
}

for url in urls_list:
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')


#------------ NAME of wine :
            
            name_w = soup.find('h1').get_text()
            dict_results['name_w'].append(name_w)
            print(f"Título de la página ({url}): {name_w}")

 #------------ YEAR of wine :
            
            year_w = soup.find('span', class_='heading fs-2em').text if soup.find('span', class_='heading fs-2em') else "No se encontró el elemento year"
            dict_results['year_w'].append(year_w)

#------------ PRICE of wine :
            
            price = soup.find('p', class_='price uniq large').text if soup.find('p', class_='price uniq large') else "No se encontró el elemento price"
            dict_results['price'].append(price)

#------------ USER SCORE:
            
            user_score = soup.find('span', class_='styles_grade__EU5A1').text if soup.find('span', class_='styles_grade__EU5A1') else "No se encontró el elemento user_score"
            dict_results['user_score'].append(user_score) 

#------------ TYPE of wine:
            
            # Encontrar la tabla con la clase especificada
            tabla = soup.find('table', class_='styles_table__jWPKo info-grid technical-info')

            if tabla:
                # Encontrar el primer elemento 'td' dentro de la tabla
                primer_td = tabla.find('td')
                
                if primer_td:
                    # Extraer el texto del primer 'td' (puede ser un enlace 'a' o texto plano)
                    type_w = primer_td.get_text(strip=True)
                    print(type_w)
                    dict_results["type_w"].append(type_w)
                else:
                    print("No se encontró el elemento 'td' dentro de la tabla")
            else:
                print("No se encontró la tabla con la clase especificada")


#------------ REGION:
                
            # Encontrar el div con la clase 'region with link'
            div_region = soup.find('div', class_='region with-link')

            if div_region:
                # Encontrar el primer elemento 'a' dentro del div
                region = div_region.find('a')
                
                if region:
                    # Imprimir el texto del primer 'a' dentro del div
                    dict_results['region'].append(region.text) 
                    print(region.text)
                else:
                    print("No se encontró el elemento 'a' dentro del div con clase 'region with link'")
            else:
                print("No se encontró el div con clase 'region with link'")

#------------ ALCOHOL content:
                

            # Encontrar la tabla con la clase especificada
            table_info = soup.find('table', class_='styles_table__jWPKo info-grid technical-info')

            if table_info:
                # Encontrar todos los elementos 'tr' dentro de la tabla
                rows = table_info.find_all('tr')
                
                # Verificar si hay suficientes filas antes de intentar acceder al sexto tr
                if len(rows) >= 6:
                    sexto_tr = rows[5]  # El índice 5 corresponde al sexto elemento (el índice inicia en 0)
                    
                    # Encontrar el primer elemento 'td' dentro del sexto 'tr'
                    tds = sexto_tr.find_all('td')
                    
                    if len(tds) > 0:
                        texto_alcohol_content = tds[0].get_text(strip=True)  # Obtener el texto del primer 'td'
                        dict_results['alcohol_content'].append(texto_alcohol_content)
                    else:
                        dict_results['alcohol_content'].append('NA')  # Agregar 'NA' si no se encontraron elementos 'td' dentro del sexto 'tr'
                else:
                    print("No hay suficientes filas para acceder al sexto tr")
                    dict_results['alcohol_content'].append('NA')
            else:
                print("No se encontró la tabla con la clase especificada")
                dict_results['alcohol_content'].append('NA')




#------------ Wine Grape:
            div_alcohol = soup.find('div', class_='tags with-link')
            
            if div_alcohol:
                wine_grape = div_alcohol.find('a')

                if wine_grape:
                    dict_results['wine_grape'].append(wine_grape.text)
                else: 
                    print("No hay info de uva")
                    dict_results['wine_grape'].append("NA")

            else:
                print("no hay div de uva")

#------------ PK score 
            tabla = soup.find('table', class_='styles_table__jWPKo info-grid score-awards')

            if tabla:
                filas = tabla.find_all('tr')
                
                found_score = ""  # Valor por defecto si no se encuentra la puntuación
                
                for fila in filas:
                    td_parker = fila.find('td', title='Parker')
                    if td_parker and td_parker.get_text(strip=True) != '':
                        found_score = td_parker.get_text(strip=True)
                        break  # Terminar el bucle después de encontrar el primer valor
                
                # Agregar 'NA' si el valor es vacío
                dict_results['pk_score'].append(found_score if found_score else "NA")
                print(f"El primer valor de Parker es: {found_score}")
            else:
                print("No se encontró la tabla con la clase especificada")
                dict_results['pk_score'].append("NA")

#------------PN score
                
            tabla = soup.find('table', class_='styles_table__jWPKo info-grid score-awards')

            if tabla:
                filas = tabla.find_all('tr')
                
                found_score = ""  # Valor por defecto si no se encuentra la puntuación
                
                for fila in filas:
                    td_penin = fila.find('td', title='Peñín')
                    if td_penin and td_penin.get_text(strip=True) != '':
                        found_score = td_penin.get_text(strip=True)
                        break  # Terminar el bucle después de encontrar el primer valor
                
                # Agregar 'NA' si el valor es vacío
                dict_results['pn_score'].append(found_score if found_score else "NA")
                print(f"El primer valor de Peñín es: {found_score}")
            else:
                print("No se encontró la tabla con la clase especificada")
                dict_results['pn_score'].append("NA")

                
#------------ SK score
                
            tabla = soup.find('table', class_='styles_table__jWPKo info-grid score-awards')

            if tabla:
                filas = tabla.find_all('tr')
                
                found_score = ""  # Valor por defecto si no se encuentra la puntuación
                
                for fila in filas:
                    td_suckling = fila.find('td', title='Suckling')
                    if td_suckling and td_suckling.get_text(strip=True) != '':
                        found_score = td_suckling.get_text(strip=True)
                        break  # Terminar el bucle después de encontrar el primer valor
                
                # Agregar 'NA' si el valor es vacío
                dict_results['sk_score'].append(found_score if found_score else "NA")
                print(f"El primer valor de Suckling es: {found_score}")
            else:
                print("No se encontró la tabla con la clase especificada")
                dict_results['sk_score'].append("NA")
                
    

            # ... FINAL
        else:
            print(f"No se pudo acceder a la URL: {url}")


    except Exception as e:
        print(f"Ocurrió un error al acceder a la URL ({url}): {str(e)}")


Título de la página (https://www.vinissimus.com/es/vino/ultreia-saint-jacques/): Ultreia Saint Jacques
Vino tinto
Bierzo
El primer valor de Parker es: 93 PK
El primer valor de Peñín es: 92 PN
El primer valor de Suckling es: 
Título de la página (https://www.vinissimus.com/es/vino/jose-pariente-verdejo/): José Pariente Verdejo
Vino blanco
Rueda
El primer valor de Parker es: 
El primer valor de Peñín es: 92 PN
El primer valor de Suckling es: 
Título de la página (https://www.vinissimus.com/es/vino/tinto-pesquera-crianza/): Tinto Pesquera Crianza
Vino tinto
Ribera del Duero
El primer valor de Parker es: 
El primer valor de Peñín es: 92 PN
El primer valor de Suckling es: 93 SK
Título de la página (https://www.vinissimus.com/es/vino/la-montesa/): La Montesa
Vino tinto
Rioja
El primer valor de Parker es: 93 PK
El primer valor de Peñín es: 93 PN
El primer valor de Suckling es: 94 SK
Título de la página (https://www.vinissimus.com/es/vino/terras-gauda/): Terras Gauda
Vino blanco
Rías Baixas
El

In [10]:
print(dict_results)

{'name_w': ['Ultreia Saint Jacques', 'José Pariente Verdejo', 'Tinto Pesquera Crianza', 'La Montesa', 'Terras Gauda', 'Ramón Bilbao Crianza', 'Viña Tondonia Reserva', 'Corimbo', 'Aalto', 'Malpaso', 'El Castro de Valtuille Mencía Joven', 'Pago de Valdoneje Mencía', 'PSI', 'Louro', 'Muga Crianza ', 'Viña Ardanza Reserva', "Les Cousins L'Inconscient", 'Paixar', 'Pétalos del Bierzo', 'Pepe Yllera Roble', 'Parada de Atauta', 'Recaredo Terrers Brut Nature', 'Les Terrasses', 'Ultreia Godello', 'Victorino', 'Almirez', 'Gramona Imperial', 'Juan Gil Etiqueta Plata', 'Lindes de Remelluri Viñedos de Labastida', 'O Luar Do Sil', 'Marqués de Murrieta', 'Els Pics', 'Cutio Garnacha', 'Carmelo Rodero Crianza', 'Muga Reserva Selección Especial', 'Rigol Brut Nature', 'Pruno', 'Raventós i Blanc de Nit', 'Valldolina Reserva Brut Nature', 'Mauro', 'Emilio Moro Finca Resalso', 'Camins del Priorat', 'Dominio del Águila Reserva', 'Tomás Postigo 3er Año', 'Matarromera Crianza', 'Juvé & Camps Reserva de la Famil

In [11]:
len(dict_results['alcohol_content'])

824

In [12]:
df_results = pd.DataFrame(dict_results)
df_results

,name_w,year_w,price,user_score,type_w,region,alcohol_content,wine_grape,pk_score,pn_score,sk_score
0,Ultreia Saint Jacques,2021,"11,50 €","4,2",Vino tinto,Bierzo,"13,5%",Mencía,93 PK,92 PN,NA
1,José Pariente Verdejo,2022,"10,50 €","4,5",Vino blanco,Rueda,"13,0%",Verdejo,NA,92 PN,NA
2,Tinto Pesquera Crianza,2020,"19,30 €","4,6",Vino tinto,Ribera del Duero,"14,5%",Tempranillo,NA,92 PN,93 SK
3,La Montesa,2020,"15,35 €","4,3",Vino tinto,Rioja,"14,0%",Garnacha,93 PK,93 PN,94 SK
4,Terras Gauda,2022,"15,75 €","4,6",Vino blanco,Rías Baixas,"12,0%",Albariño,NA,92 PN,NA
...,...,...,...,...,...,...,...,...,...,...,...
819,Capricho Val de Paxariñas,2022,"11,95 €","4,8",Vino blanco,Bierzo,"13,5%",Godello,90 PK,93 PN,NA
820,Campo Viejo Crianza,2020,"5,75 €","4,5",Vino tinto,Rioja,"13,5%",Tempranillo,NA,NA,NA
821,Poesia Garnacha Tinta,2021,"3,90 €","4,1",Vino tinto,Catalunya,"13,5%",Garnacha,NA,NA,NA
822,Monasterio de las Viñas Crianza,2019,"5,20 €","5,0",Vino tinto,Cariñena,"13,5%",Tempranillo,NA,NA,NA


In [13]:
df_results.to_csv("1_table_wines_info_bsoup.csv")